In [1]:
import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from fireworks import LaunchPad
import os
from atomate.vasp.powerups import add_modify_incar, add_modify_potcar, add_tags
from ph3pywf.workflows.core import wf_phono3py
from ph3pywf.workflows.core import wf_disp_from_optimized

successfully loaded your custom FW_config.yaml!


/opt/conda/lib/python3.8/site-packages/maggma/utils.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Full phono3py workflow
see below

In [2]:
####################
# FULL PHONO3PY WF #
####################
### Running conditions ##############################################
material_id = "mp-2574" # Material id to be searched on MP
supercell_size = (2,2,2)
cutoff_pair_distance = 5.0
is_reduced_test = True

### Get working dir #################################################
working_dir = os.getcwd()

### Get materials structure #########################################
## If no local structure file, get from MP and save unitcell
## structure as "struct_unitcell.json"
if "struct_unitcell.json" in os.listdir(working_dir):
    print("Found local structure file")
    ## Read saved local structure file
    struct_unitcell = Structure.from_file("struct_unitcell.json")

else:
    print("Getting structure from MaterialsProject")
    # api_key = ''
    with MPRester() as mpr:
        struct_unitcell = mpr.get_structure_by_material_id(material_id)

    ## Save a local structure file to avoid accessing MP every time
    with open("struct_unitcell.json", "w") as fh:
        fh.write(struct_unitcell.to_json())

### Create the workflow #############################################
c = {"supercell_size": supercell_size, 
     "cutoff_pair_distance": cutoff_pair_distance, 
     "is_reduced_test": is_reduced_test, 
     "USER_INCAR_SETTINGS": {'ENCUT': 520,
                             'ALGO': 'Normal'
                            },
     "USER_POTCAR_SETTINGS": {'Zr': 'Zr_sv'},
     "USER_POTCAR_FUNCTIONAL": "PBE_54",
     "metadata": ["test reduced phono3py wf"]
     }

print("Creating workflow...")
workflow = wf_phono3py(structure=struct_unitcell, 
                       c=c,
                      )
print("Created workflow")

### Add Powerups ####################################################


### Create the launchpad and add our workflow #######################
print("Sending to LaunchPad...")
launchpad = LaunchPad.from_file("/home/jovyan/atomate/config/my_launchpad.yaml")
launchpad.add_wf(workflow)
print("Sent to LaunchPad")
print(">>Done<<")

Found local structure file
Creating workflow...
Created workflow
Sending to LaunchPad...


/opt/conda/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:440: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/pymatgen/io/vasp/sets.py:451: BadInputSetWarning: Overriding POTCARs is generally not recommended as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. In many instances, it is better to write a subclass of a desired input set and override the POTCAR in the subclass to be explicit on the differences.
  warnings.warn(


2021-05-26 04:58:04,975 INFO Added a workflow. id_map: {-2: 1806, -1: 1807}
Sent to LaunchPad
>>Done<<


# Testing workflow using existing optimized structure
see below

In [ ]:
### Running conditions ##############################################
material_id = "mp-2574" # Material id to be searched on MP
supercell_size = (2,2,2)

### Get working dir #################################################
working_dir = os.getcwd()
# db_file = os.path.join(working_dir, "db.json")

### Get materials structure #########################################
## If no local structure file, get from MP and save unitcell
## structure as "struct_unitcell.json"
if "struct_unitcell.json" in os.listdir(working_dir):
    print("Found local structure file")
    ## Read saved local structure file
    struct_unitcell = Structure.from_file("struct_unitcell.json")

else:
    print("Getting structure from MaterialsProject")
    # api_key = ''
    with MPRester() as mpr:
        struct_unitcell = mpr.get_structure_by_material_id(material_id)

    ## Save a local structure file to avoid accessing MP every time
    with open("struct_unitcell.json", "w") as fh:
        fh.write(struct_unitcell.to_json())

### Create the workflow #############################################
print("Creating workflow...")
workflow = wf_disp_from_optimized(structure=struct_unitcell, 
                                  supercell_size=supercell_size, 
                                 )
print("Created workflow")

### Create the launchpad and add our workflow #######################
print("Sending to LaunchPad...")
launchpad = LaunchPad.from_file("/home/jovyan/atomate/config/my_launchpad.yaml")
launchpad.add_wf(workflow)
print("Sent to LaunchPad")